In [59]:
from functools import reduce

In [60]:
with open("data_inputs/day16_input.txt") as f:
    input = f.read()

In [61]:
d = """0 = 0000
1 = 0001
2 = 0010
3 = 0011
4 = 0100
5 = 0101
6 = 0110
7 = 0111
8 = 1000
9 = 1001
A = 1010
B = 1011
C = 1100
D = 1101
E = 1110
F = 1111"""

d_l = d.split("\n")
h2b = {dd[0]: dd[4:8] for dd in d_l}
b2h = {dd[4:8]: dd[0] for dd in d_l}

id2op = {
    "000": "sum", 
    "001": "mul",
    "010": "min",
    "011": "max",
    "101": "gt", 
    "110": "lt",
    "111": "eq",
}


In [62]:
def hex2bin(input):

    output = []
    for _i in input:
        output.append(h2b[_i])

    return "".join(output)


def bin2hex(input):
    
    output = []
    for _i in range(0, len(input), 4):
        output.append(b2h[input[_i:_i+4]])
        print(input[_i:_i+4])

    return "".join(output)

In [63]:
input_bin = hex2bin(input)

In [64]:
def operate(op, literals):
    if op == "000":
        return sum(literals)

    elif op == "001":
        return reduce((lambda x, y: x * y), literals)

    elif op == "010":
        return min(literals)
    
    elif op == "011":
        return max(literals)

    elif op == "101":
        return 1 * (literals[0] > literals[1])
    
    elif op == "110":
        return 1 * (literals[0] < literals[1])

    elif op == "111":
        return 1 * (literals[0] == literals[1])


In [65]:
def next_packet(bit0, versions, operation, input_bin, level):

    versions.append(int(input_bin[bit0:(bit0+3)], 2))

    # Packet
    if input_bin[(bit0+3):(bit0+6)] == '100':
        
        zero = False
        bit0_literal = bit0+6
        literal = []
        while not zero:
            if input_bin[bit0_literal] == '0':
                literal.append(b2h[input_bin[bit0_literal+1:bit0_literal+5]])
                zero = True
            else:
                literal.append(b2h[input_bin[bit0_literal+1:bit0_literal+5]])
                bit0_literal += 5
        
        bitf_literal = bit0_literal + 5
        dec_literal = int("".join(literal), 16)
        operation.append((level, dec_literal))

        return bitf_literal, dec_literal

    # Operator
    else:
        operator_id = input_bin[(bit0+3):(bit0+6)]

        if input_bin[bit0+6] == '0':
            subpacket_len = int(input_bin[(bit0+7):(bit0+22)], 2)
            l = 0
            sub_bit0 = bit0+22
            sub_bitf = None
            literals = []

            while l < subpacket_len:
                sub_bitf, literal = next_packet(sub_bit0, versions, operation, input_bin, level+1)
                literals.append(literal)
                l += sub_bitf - sub_bit0
                sub_bit0 = sub_bitf
            
            result = operate(operator_id, literals)
            operation.append((level, id2op[operator_id], literals, result))

            return sub_bitf, result
        
        else: 
            num_subpackets = int(input_bin[(bit0+7):(bit0+18)], 2)
            sub_bit0 = bit0+18
            sub_bitf = None
            literals = []  

            for sp in range(num_subpackets):
                sub_bitf, literal = next_packet(sub_bit0, versions, operation, input_bin, level+1)
                literals.append(literal)
                sub_bit0 = sub_bitf

            result = operate(operator_id, literals)
            operation.append((level, id2op[operator_id], literals, result))

            return sub_bitf, result

In [66]:
versions = []
operation = []
bitf = 0

_, result2 = next_packet(bitf, versions, operation, input_bin, 0)

print("Result1:", sum(versions))
print("Result2:", result2)
            



Result1: 949
Result2: 1114600142730
